In [1]:
import ee
import os

service_account = 'jupyter-gee-project@ee-avs-dse.iam.gserviceaccount.com'
key_path = '../service_account_key.json'

credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials)

print(ee.String('Hello from the Earth Engine servers!').getInfo())

Hello from the Earth Engine servers!


In [ ]:
def remove_high_perimeter_area_ratio(features):
    """Calculate perimeter to area ratio and filter out narrow features in top 25%"""
    features_with_ratio = features.map(lambda f: f.set({
        'perimeter_area_ratio': f.geometry().perimeter().divide(f.get('GIS_AREA'))
    }))
    ratio_stats = features_with_ratio.aggregate_stats('perimeter_area_ratio')
    ratio_threshold = ratio_stats.get('p75')
    narrow_filter = ee.Filter.lt('perimeter_area_ratio', ratio_threshold)
    
    return features_with_ratio, narrow_filter

# Load protected areas dataset
protected_areas = ee.FeatureCollection("WCMC/WDPA/202106/polygons")

# Define filters
marine_filter = ee.Filter.eq("MARINE", "0")
not_mpa_filter = ee.Filter.neq("DESIG_ENG", "Marine Protected Area")
status_filter = ee.Filter.inList("STATUS", ["Designated", "Established", "Inscribed"])
designation_filter = ee.Filter.neq("DESIG_ENG", "UNESCO-MAB Biosphere Reserve")
excluded_pids = ["555655917", "555656005", "555656013", "555665477", "555656021",
                 "555665485", "555556142", "187", "555703455", "555563456", "15894"]
area_filter = ee.Filter.gte("GIS_AREA", 200)

# Filter to exclude specific WDPA_PID values
pids_filter = ee.Filter.inList("WDPA_PID", excluded_pids).Not()

# Combine all filters
combined_filter = ee.Filter.And(
    marine_filter,
    not_mpa_filter,
    status_filter,
    designation_filter,
    pids_filter,
    area_filter
)

# Apply filters
data = protected_areas.filter(combined_filter)

# Get WDPA_PIDs
wdpa_pids = data.aggregate_array("WDPA_PID").getInfo()
print(f"Number of protected areas: {len(wdpa_pids)}")

In [2]:
import ee
import geopandas as gpd
from math import ceil
import time

def get_narrow_protected_area_ids(batch_size=500):
    """Get WDPA IDs of narrow protected areas (top 25% perimeter-area ratio)"""
    
    # Load and filter protected areas
    protected_areas = ee.FeatureCollection("WCMC/WDPA/202106/polygons")
    filtered_pas = protected_areas.filter(ee.Filter.And(
        ee.Filter.eq("MARINE", "0"),
        ee.Filter.neq("DESIG_ENG", "Marine Protected Area"),
        ee.Filter.inList("STATUS", ["Designated", "Established", "Inscribed"]),
        ee.Filter.neq("DESIG_ENG", "UNESCO-MAB Biosphere Reserve"),
        ee.Filter.gte("GIS_AREA", 200), 
        ee.Filter.inList("WDPA_PID", [
            "555655917", "555656005", "555656013", "555665477", "555656021",
            "555665485", "555556142", "187", "555703455", "555563456", "15894"
        ]).Not()
    ))
    
    # Process in batches to handle size limits
    total = filtered_pas.size().getInfo()
    batches = ceil(total / batch_size)
    print(f"Processing {total} protected areas in {batches} batches...")
    
    all_features = []
    for i in range(batches):
        start = i * batch_size
        batch = filtered_pas.toList(batch_size, start)
        batch_fc = ee.FeatureCollection(batch)
        features = batch_fc.getInfo()['features']
        all_features.extend(features)
        print(f"Processed batch {i+1}/{batches}")
        time.sleep(1)  # Pause to avoid rate limits
    
    # Calculate ratios and get IDs of narrow areas
    gdf = gpd.GeoDataFrame.from_features(all_features)
    gdf['perimeter_area_ratio'] = gdf.geometry.length / gdf.GIS_AREA
    threshold = gdf.perimeter_area_ratio.quantile(0.75)
    narrow_ids = gdf[gdf.perimeter_area_ratio >= threshold].WDPA_PID.tolist()
    
    print(f"Found {len(narrow_ids)} narrow protected areas")
    return narrow_ids

# Get IDs of narrow protected areas
narrow_protected_area_ids = get_narrow_protected_area_ids()

# Save IDs to file for future use
with open('filtered_pas.txt', 'w') as f:
    f.write('\n'.join(map(str, narrow_protected_area_ids)))

print("Narrow protected area IDs saved to narrow_protected_area_ids.txt")

Processing 9272 protected areas in 19 batches...
Processed batch 1/19
Processed batch 2/19
Processed batch 3/19
Processed batch 4/19
Processed batch 5/19
Processed batch 6/19
Processed batch 7/19
Processed batch 8/19
Processed batch 9/19
Processed batch 10/19
Processed batch 11/19


EEException: Response size exceeds limit

In [ ]:
import geopandas as gpd
from shapely.ops import unary_union

def get_outermost_polygon(gdf, target_crs="ESRI:54009"):
    """
    Returns a GeoDataFrame with the outer boundary of all input geometries.

    Args:
        gdf (GeoDataFrame): Input polygons.
        target_crs (str): Target CRS for equal-area measurements.

    Returns:
        GeoDataFrame: Single outermost polygon as a GeoDataFrame.
    """
    gdf_proj = gdf.to_crs(target_crs)
    merged = unary_union(gdf_proj.geometry)
    
    # If multiple disjoint pieces remain, wrap them in MultiPolygon
    return gpd.GeoDataFrame(geometry=[merged], crs=target_crs)


In [ ]:
# Merge all polygons into a single geometry
merged_geom = unary_union(cleaned.geometry)

# Optional: buffer inward to remove dilation effect
shrink_dist = -((hole_pix * resolution) / 2)
merged_geom = merged_geom.buffer(shrink_dist)

# Wrap in GeoDataFrame
outer = gpd.GeoDataFrame(geometry=[merged_geom], crs=target_crs)


First, install PostgreSQL and PostGIS:
Start PostgreSQL service:
Create a new database and enable PostGIS:
Import WDPA data from GeoJSON (assuming you have downloaded it from Earth Engine):
Once you have the GeoJSON file, import it to PostGIS:
Calculate perimeter-area ratios:
Use the results in your Earth Engine code:
narrow_filter = ee.Filter.inList('WDPA_PID', filtered_ids['wdpa_pid'].tolist())
